Q1: Consider only the below columns and prepare a prediction model for predicting Price.

Corolla<-Corolla[c("Price","Age_08_04","KM","HP","cc","Doors","Gears","Quarterly_Tax","Weight")]

Do transformations for getting better predictions of profit and make a table containing R^2 value for each prepared model.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
from statsmodels.graphics.regressionplots import influence_plot
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
toyota = pd.read_csv('/Users/nihadnazar/Desktop/ASSIGNMENTS/DATASETS/ToyotaCorolla.csv',encoding='latin1')
toyota

EDA

In [ ]:
print(toyota.info())

In [ ]:
toyota = toyota.rename({'Quarterly_Tax':'QT','Age_08_04':'AGE','Price':'PRICE','cc':'CC','Doors':'DOORS','Gears':'GEARS','Weight':'WEIGHT'},axis = 1)
toyota[["PRICE","AGE","KM","HP","CC","DOORS","GEARS","QT","WEIGHT"]]

In [ ]:
toyota[toyota.duplicated()]

In [ ]:
toyota=toyota.drop_duplicates().reset_index(drop=True)
toyota[["PRICE","AGE","KM","HP","CC","DOORS","GEARS","QT","WEIGHT"]]

In [ ]:
toyota[["PRICE","AGE","KM","HP","CC","DOORS","GEARS","QT","WEIGHT"]].describe

CORRELATION ANALYSIS

In [ ]:
toyota[["PRICE","AGE","KM","HP","CC","DOORS","GEARS","QT","WEIGHT"]].corr()

In [ ]:
sns.set_style(style = 'whitegrid')
sns.pairplot(toyota[["PRICE","AGE","KM","HP","CC","DOORS","GEARS","QT","WEIGHT"]])

MODEL BUILDING

In [ ]:
model =smf.ols("PRICE ~ AGE+KM+HP+CC+DOORS+GEARS+QT+WEIGHT",data = toyota).fit()

MODEL TESTING

In [ ]:
model.params

In [ ]:
model.tvalues , np.round(model.pvalues,5)

In [ ]:
model.rsquared , model.rsquared_adj

In [ ]:
#BUILDING SLR AND MLR MODEL FOR DOOR AND CC
slr_cc = smf.ols(formula = 'PRICE ~ CC',data=toyota).fit()
print('P VALUE OF CC: \n',slr_cc.pvalues,'\n\nT VALUE OF CC: \n',slr_cc.tvalues)

In [ ]:
slr_d = smf.ols(formula = 'PRICE ~ DOORS',data = toyota).fit()
print('P VALUE OF DOORS: \n',slr_d.pvalues,'\n\nT VALUE OF DOORS: \n',slr_d.tvalues)

In [ ]:
mlr_cc_d = smf.ols(formula = 'PRICE ~ CC+DOORS',data = toyota).fit()
print('P VALUE OF CC AND DOORS: \n',mlr_cc_d.pvalues,'\n\nT VALUE OF CC AND DOORS: \n',mlr_cc_d.tvalues)

AS WE CAN SEE,

HERE THE CC AND DOORS HAVE A SIGNIFICANT P VALUE IN CASE OF SLR AND MLR.

MODEL VALIDATION TECHNIQUES

THERE ARE TWO TECHINQUES:

    • COLINEARITY CHECK

    • RESIDUAL ANALYSIS

In [ ]:
# COLINEARITY CHECKING
# VIF = 1/(1 - RSQUARE) OF EACH INDEPENDENT VARIABLES

rsq_age = smf.ols('AGE ~ KM+HP+CC+DOORS+GEARS+QT+WEIGHT',data = toyota).fit().rsquared
vif_age = 1 / (1 - rsq_age)

rsq_km = smf.ols('KM ~ AGE+HP+CC+DOORS+GEARS+QT+WEIGHT ',data = toyota).fit().rsquared
vif_km = 1 / (1 - rsq_km)

rsq_hp = smf.ols('HP ~ AGE+KM+CC+DOORS+GEARS+QT+WEIGHT ',data = toyota).fit().rsquared
vif_hp = 1 / (1 - rsq_hp)

rsq_cc = smf.ols('CC ~ AGE+KM+HP+DOORS+GEARS+QT+WEIGHT ',data = toyota).fit().rsquared
vif_cc = 1 / (1 - rsq_cc)

rsq_doors = smf.ols('DOORS ~ AGE+KM+HP+CC+GEARS+QT+WEIGHT ',data = toyota).fit().rsquared
vif_doors = 1 / (1 - rsq_doors)

rsq_gears = smf.ols('GEARS ~ AGE+KM+HP+CC+DOORS+QT+WEIGHT ',data = toyota).fit().rsquared
vif_gears = 1 / (1 - rsq_gears)

rsq_qt = smf.ols('QT ~ AGE+KM+HP+CC+DOORS+GEARS+WEIGHT ',data = toyota).fit().rsquared
vif_qt = 1 / (1 - rsq_qt)

rsq_weight = smf.ols('WEIGHT ~ AGE+KM+HP+CC+DOORS+GEARS+QT ',data = toyota).fit().rsquared
vif_weight = 1 / (1 - rsq_weight)

df = {'VARIABLES':["AGE","KM","HP","CC","DOORS","GEARS","QT","WEIGHT"],'VIF' : [vif_age,vif_km,vif_hp,vif_cc,vif_doors,vif_gears,vif_qt,vif_weight]}
VIF_DF = pd.DataFrame(df)
VIF_DF

In [ ]:
# so we can consider all the variables in the regression equation since there is no vif>20
# 2. RESIDUAL ANALYSIS

import statsmodels.api as sm
sm.qqplot(model.resid,line = 'q') 
# 'q' - A line is fit through the quartiles # line = '45'- to draw the 45-degree diagonal line
plt.title('NORMAL QQ-PLOT OF RESIDUALS')
plt.show()

In [ ]:
# To detect the residuals in qq-plot
list(np.where(model.resid>6000))

In [ ]:
list(np.where(model.resid<-6000))

In [ ]:
#TEST FOR HOMOSCEDASTICITY OR HETEROSCEDASTICITY
def standard_values(vals) : 
    return (vals - vals.mean())/vals.std() # z = (x - µ) / sigma

In [ ]:
plt.scatter(standard_values(model.fittedvalues),standard_values(model.resid))
plt.title('RESIDUAL PLOTS')
plt.xlabel('STANDARDIZED FITTED VALUES')
plt.ylabel('STANDARDIZED RESIDUALS VALUES')
plt.show()

In [ ]:
# TEST FOR ERROR OR RESIDUAL VS REGRESSORS
fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'AGE',fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'KM',fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'HP',fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'CC',fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'DOORS',fig=fig)
plt.show()


In [ ]:
fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'GEARS',fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'QT',fig=fig)
plt.show()


In [ ]:
fig = plt.figure(figsize=(15,8))
sm.graphics.plot_regress_exog(model,'WEIGHT',fig=fig)
plt.show()

MODEL DELETION DIAGNOSTICS.

THERE ARE TWO TECHNIQUES:
    
    • COOK'S DISTANCE
    
    • LEVERAGE VALUE

In [ ]:
#COOK'S DISTANCE
model_influence = model.get_influence()
(c,_) = model_influence.cooks_distance

In [ ]:
fig = plt.subplots(figsize=(20,7))
plt.stem(np.arange(len(toyota)),np.round(c,3))
plt.xlabel('ROW INDEX')
plt.ylabel('COOKS DISTANCE')
plt.show()

In [ ]:
(np.argmax(c),np.max(c))

In [ ]:
# 2. LEVERAGE VALUE
fig,ax = plt.subplots(figsize=(20,20))
fig = influence_plot(model,ax=ax)

In [ ]:
# leverage cut-off value = 3 * (k+1)/n ; k = no. of features/columns and n = no. of datapoints
k = toyota.shape[1]
n = toyota.shape[0]
leverage_cutoff = 3*(k+1)/n
leverage_cutoff

In [ ]:
toyota[toyota.index.isin([80])]

In [ ]:
new = toyota.copy()
new

In [116]:
toyota_1 = new.drop(new.index[[50]],axis = 0).reset_index(drop = True)
toyota_1

,Id,Model,PRICE,AGE,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Central_Lock,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,1,0,0,0,1,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,1,0,1,0,0,0,1,0,0,0
2,3,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13950,24,9,2002,41711,Diesel,90,1,...,0,0,1,0,0,0,1,0,0,0
3,4,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,14950,26,7,2002,48000,Diesel,90,0,...,0,0,1,0,0,0,1,0,0,0
4,5,TOYOTA Corolla 2.0 D4D HATCHB SOL 2/3-Doors,13750,30,3,2002,38500,Diesel,90,0,...,1,1,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430,1438,TOYOTA Corolla 1.3 16V HATCHB G6 2/3-Doors,7500,69,12,1998,20544,Petrol,86,1,...,1,1,1,0,1,1,1,0,0,0
1431,1439,TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...,10845,72,9,1998,19000,Petrol,86,0,...,0,0,1,0,0,1,1,0,0,0
1432,1440,TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...,8500,71,10,1998,17016,Petrol,86,0,...,0,0,1,0,0,0,1,0,0,0
1433,1441,TOYOTA Corolla 1.3 16V HATCHB LINEA TERRA 2/3-...,7250,70,11,1998,16916,Petrol,86,1,...,0,0,0,0,0,0,1,0,0,0


In [119]:
final_model = smf.ols('PRICE~AGE+KM+HP+CC+DOORS+GEARS+QT+WEIGHT',data = toyota).fit()
final_model.rsquared

0.8637627463428192

In [ ]:
toyota_1

MODEL PREDICTIONS

In [118]:
new_data = pd.DataFrame({'AGE':10,'KM':38000,'HP':80,'CC':1300,'DOORS':4,'GEARS':5,'QT':70,'WEIGHT':1000},index=[0])
new_data

,AGE,KM,HP,CC,DOORS,GEARS,QT,WEIGHT
0,10,38000,80,1300,4,5,70,1000


In [120]:
final_model.predict(new_data)

0    14996.501402
dtype: float64

In [121]:
prediction = final_model.predict(toyota_1)
prediction

0     16812.580505
1     16272.355186
2     16800.732460
3     16426.496731
4     16222.419010
          ...     
75    15883.099082
76    16152.345250
77    15928.055303
78    15345.941875
79    15885.600444
Length: 80, dtype: float64